In [1]:
import pandas as pd
import os
import sys
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parents[1]))

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold,RandomizedSearchCV

from src.variables import target, id_var, categoric, numeric
from src.feature_engineering import * 


## Read clean data

In [ ]:
app_train, app_test = process_categoric(app_train_clean,app_test_clean, categoric)
app_train.to_pickle('/Credit-default-risk/data/app_train.p')
app_test.to_pickle('/Credit-default-risk/data/app_test.p')

In [3]:
app_train = pd.read_pickle('/Credit-default-risk/data/app_train.p')

In [103]:
app_test = pd.read_pickle('/Credit-default-risk/data/app_test.p')

## Split train

In [4]:
X_train, X_test, y_train, y_test = train_test_split(app_train.drop(target+id_var, axis = 1), app_train[target], test_size=0.33, random_state=42)

In [5]:
train,test, features = impute_scale(X_train, X_test)

## Impute with median and Scale data

In [109]:
train_test,test_real, features_test = impute_scale(X_train, app_test.drop(id_var, axis = 1))

In [110]:
test_real = pd.DataFrame(test_real, columns = features_test)

In [6]:
train = pd.DataFrame(train, columns = features)
test = pd.DataFrame(test, columns = features)

In [7]:
train_target = train.copy()
train_target['target'] = y_train.values

## Search correlation

In [8]:
cor = train_target.corr().abs()

In [9]:
to_rm = list(cor[(cor!=1) & (cor>0.5)].sum()[cor[(cor!=1) & (cor>0.5)].sum()>0].index)

In [10]:
cor[['target']].loc[to_rm].sort_values('target',ascending = False).tail(20).index

Index(['landarea_avg', 'landarea_mode', 'organization_type_medicine',
       'cnt_fam_members', 'name_income_type_commercial_associate',
       'occupation_type_medicine_staff', 'obs_30_cnt_social_circle',
       'obs_60_cnt_social_circle', 'name_type_suite_unaccompanied',
       'name_type_suite_family', 'reg_region_not_work_region',
       'organization_type_security', 'years_beginexpluatation_medi',
       'nonlivingapartments_avg', 'years_beginexpluatation_avg',
       'nonlivingapartments_medi', 'years_beginexpluatation_mode',
       'nonlivingapartments_mode', 'live_region_not_work_region',
       'name_housing_type_municipal_apartment'],
      dtype='object')

In [11]:
to_rm = [ 'landarea_mode', 'organization_type_medicine',
       'cnt_fam_members', 'name_income_type_commercial_associate',
       'occupation_type_medicine_staff', 'obs_30_cnt_social_circle',
       'obs_60_cnt_social_circle', 
       'name_type_suite_family', 'reg_region_not_work_region',
       'organization_type_security', 'years_beginexpluatation_avg',
       'nonlivingapartments_medi', 'years_beginexpluatation_mode',
       'nonlivingapartments_mode', 'live_region_not_work_region',
       'name_housing_type_municipal_apartment', 'livingapartments_medi', 'commonarea_avg',
        'entrances_medi']

In [29]:
cor['target'].sort_values(ascending = False).head(20).index

Index(['target', 'ext_source_2', 'ext_source_3', 'ext_source_1', 'days_birth',
       'region_rating_client_w_city', 'region_rating_client',
       'name_income_type_working', 'name_education_type_higher_education',
       'code_gender_m', 'code_gender_f', 'days_last_phone_change',
       'days_id_publish', 'reg_city_not_work_city',
       'name_education_type_secondary_/_secondary_special',
       'name_income_type_pensioner', 'flag_emp_phone', 'organization_type_xna',
       'reg_city_not_live_city', 'days_employed'],
      dtype='object')

In [12]:
train.drop(to_rm, axis = 1, inplace = True)
test.drop(to_rm, axis = 1, inplace = True)
train_target.drop(to_rm, axis = 1, inplace = True)

In [14]:
xgb_model = xgb.XGBClassifier()

## Select Variables 

In [16]:
selector = RFE(xgb_model, n_features_to_select=5, step=1)
selector = selector.fit(train, y_train)

/tmp/.local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/credit/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[06:44:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/tmp/.local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/credit/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[06:45:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/tmp/.local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/credit/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[07:47:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/tmp/.local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/credit/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[07:47:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/tmp/.local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/credit/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[07:47:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [40]:
ranking = list(selector.ranking_)

In [53]:
indexes = [i for i,x in enumerate(ranking) if x <120]

In [30]:
cor_max = ['ext_source_2', 'ext_source_3', 'ext_source_1', 'days_birth',
       'region_rating_client_w_city', 'region_rating_client',
       'name_income_type_working', 'name_education_type_higher_education',
       'code_gender_m', 'code_gender_f', 'days_last_phone_change',
       'days_id_publish', 'reg_city_not_work_city',
       'name_education_type_secondary_/_secondary_special',
       'name_income_type_pensioner', 'flag_emp_phone', 'organization_type_xna',
       'reg_city_not_live_city', 'days_employed']

In [56]:
features_select = list(train.iloc[:, indexes].columns)

In [84]:
features_select

['cnt_children',
 'amt_income_total',
 'amt_credit',
 'amt_annuity',
 'amt_goods_price',
 'region_population_relative',
 'days_birth',
 'days_employed',
 'days_registration',
 'days_id_publish',
 'own_car_age',
 'flag_work_phone',
 'flag_phone',
 'flag_email',
 'region_rating_client',
 'region_rating_client_w_city',
 'hour_appr_process_start',
 'reg_city_not_live_city',
 'ext_source_1',
 'ext_source_2',
 'ext_source_3',
 'apartments_avg',
 'basementarea_avg',
 'years_build_avg',
 'entrances_avg',
 'floorsmax_avg',
 'floorsmin_avg',
 'landarea_avg',
 'livingapartments_avg',
 'livingarea_avg',
 'nonlivingapartments_avg',
 'nonlivingarea_avg',
 'apartments_mode',
 'basementarea_mode',
 'years_build_mode',
 'commonarea_mode',
 'elevators_mode',
 'entrances_mode',
 'floorsmax_mode',
 'floorsmin_mode',
 'livingapartments_mode',
 'livingarea_mode',
 'nonlivingarea_mode',
 'apartments_medi',
 'basementarea_medi',
 'years_beginexpluatation_medi',
 'years_build_medi',
 'commonarea_medi',
 'landa

## Grid Search Xgboost

In [98]:
params = {'nthread':[4], 
              'objective':['binary:logistic'],
              'learning_rate': [0.05],
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], 
              'seed': [1337]}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=params,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)



In [113]:
%%time
grid_search.fit(train_target[features_select],train_target[['target']])

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/tmp/.local/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/credit/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[08:47:27] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[08:47:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 6.87 s, sys: 1.28 s, total: 8.15 s
Wall time: 30.7 s


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_job...
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=None,
                                     verbosity=None),
  

In [114]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=11, missing=nan, monotone_constraints='()',
              n_estimators=5, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=1337, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=1337, silent=1, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

## Results

In [115]:
preds = grid_search.predict(test[features_select])

In [117]:
roc_auc_score(preds, y_test)

0.6165127462403702

In [125]:
preds_final = grid_search.predict(test_real[features_select])

In [126]:
submission = app_test[id_var]

In [127]:
submission['TARGET'] = preds_final

<ipython-input-127-426c5ff837e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['TARGET'] = preds_final


In [130]:
submission.to_csv('submission.csv', index = False)